In [ ]:
import os, glob, datetime, dill, IPython, pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Backend
class SessionStateManager:
    def __init__(self, checkpoint_dir="checkpoints", baseline_idx=0):
        self.dir = checkpoint_dir
        os.makedirs(self.dir, exist_ok=True)
        self.counter = len(glob.glob(os.path.join(self.dir, "*.pkl")))
        self.baseline_idx = baseline_idx

    def _filter_ns(self, ns):
        filt = {}
        for k,v in ns.items():
            if k.startswith("_") or k in ("In","Out"):
                continue
            try:
                dill.dumps(v)
                filt[k] = v
            except:
                pass
        return filt

    def manual_checkpoint(self, ns):
        ip   = IPython.get_ipython()
        code = ip.user_ns["In"][-1].strip()
        self.counter += 1
        ts    = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        name  = f"chk_{self.counter}_{ts}.pkl"
        path  = os.path.join(self.dir, name)
        payload = (self._filter_ns(ns), code)
        with open(path, "wb") as f:
            dill.dump(payload, f)
        return name

    def auto_checkpoint(self, ns):
        ip   = IPython.get_ipython()
        code = ip.user_ns["In"][-1].strip()
        snap = self._filter_ns(ns)
        self.counter += 1
        ts   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        name = f"auto_{self.counter}_{ts}.pkl"
        path = os.path.join(self.dir, name)
        dill.dump((snap, code), open(path, "wb"))
        return name

    def list_checkpoints(self):
        return sorted(os.listdir(self.dir))

    def restore(self, fname, ns):
        path = os.path.join(self.dir, fname)
        payload = dill.load(open(path, "rb"))
        ns_dict, _ = payload
        ns.update(ns_dict)
        ip = IPython.get_ipython()
        self.baseline_idx = len(ip.user_ns["In"]) - 1

if "_elastic_hook_registered" not in globals():
    manager = None
    def _post_run(_):
        global manager
        if not manager: return
        ip  = IPython.get_ipython()
        idx = len(ip.user_ns["In"]) - 1
        if idx <= manager.baseline_idx: return
        fname = manager.auto_checkpoint(ip.user_ns)
        print("Auto‑saved:", fname)
    IPython.get_ipython().events.register("post_run_cell", _post_run)
    _elastic_hook_registered = True

# UI 
btn_l = widgets.Layout(width="120px", margin="5px")
dd_l  = widgets.Layout(width="240px", margin="5px")

header      = widgets.HTML("<h2 style='color:#2E86C1;'>ElasticNotebook Demo</h2>")
init_btn    = widgets.Button(description="Initialize", icon="plug",  button_style="success", layout=btn_l)
chk_btn     = widgets.Button(description="Checkpoint", icon="save",  button_style="info",   layout=btn_l)
restore_btn = widgets.Button(description="Restore",    icon="undo",  button_style="warning",layout=btn_l)
view_btn    = widgets.Button(description="View",       icon="eye",   button_style="primary",layout=btn_l)
dropdown    = widgets.Dropdown(options=[""], layout=dd_l)

log_out      = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px", height="120px", overflow="auto"))
view_out     = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px", height="200px", overflow="auto"))
snippet_area = widgets.Textarea(placeholder="Code snippet…", layout=widgets.Layout(width="100%", height="80px"))

tab1 = widgets.VBox([
    widgets.HBox([init_btn, chk_btn]),
    widgets.HBox([dropdown, restore_btn]),
    widgets.Label("Status log:"), log_out,
    widgets.Label("Code snippet:"), snippet_area
], layout=widgets.Layout(padding="10px"))

tab2 = widgets.VBox([
    widgets.HBox([dropdown, view_btn]),
    widgets.Label("Checkpoint contents:"), view_out
], layout=widgets.Layout(padding="10px"))

tabs = widgets.Tab([tab1, tab2])
tabs.set_title(0, "Checkpoint/Restore")
tabs.set_title(1, "View")
display(header, tabs)

def refresh_dropdown():
    sel = dropdown.value
    opts = [""] + manager.list_checkpoints()
    dropdown.options = opts
    if sel in opts:
        dropdown.value = sel

def load_snippet(fname):
    if not fname: 
        return ""
    ns_dict, code = dill.load(open(os.path.join(manager.dir, fname), "rb"))
    return code

def on_init(_):
    global manager
    base = len(IPython.get_ipython().user_ns["In"]) - 1
    manager = SessionStateManager(baseline_idx=base)
    with log_out:
        clear_output(); print(f"Initialized. Skipping first {base} cells.")
    refresh_dropdown()
    snippet_area.value = ""

def on_checkpoint(_):
    if not manager:
        with log_out: print("Initialize first."); return
    fname = manager.manual_checkpoint(IPython.get_ipython().user_ns)
    refresh_dropdown(); dropdown.value = fname
    with log_out: print("Saved:", fname)
    snippet_area.value = load_snippet(fname)

def on_restore(_):
    if not manager:
        with log_out: print("Initialize first."); return
    fname = dropdown.value or ""
    if not fname:
        with log_out: print("Select a checkpoint."); return
    manager.restore(fname, IPython.get_ipython().user_ns)
    refresh_dropdown(); dropdown.value = fname
    with log_out: print("🔄 Restored:", fname)
    snippet_area.value = load_snippet(fname)

def on_view(_):
    if not manager:
        with view_out: print("Initialize first."); return
    fname = dropdown.value or ""
    if not fname:
        with view_out: print("Select a checkpoint."); return
    ns_dict, _ = dill.load(open(os.path.join(manager.dir, fname), "rb"))
    rows = [{"Variable":k, "Type":type(v).__name__, "Preview":repr(v)[:60]} for k,v in ns_dict.items()]
    with view_out:
        clear_output(); display(pd.DataFrame(rows))

init_btn.on_click(on_init)
chk_btn.on_click(on_checkpoint)
restore_btn.on_click(on_restore)
view_btn.on_click(on_view)

HTML(value="<h2 style='color:#2E86C1;'>ElasticNotebook Demo</h2>")

In [2]:
x=[100,200,300]

Auto‑saved: auto_1_20250420_163347.pkl
